<a href="https://colab.research.google.com/github/Sianwa/OpenAIGym---Taxi-Game/blob/master/Taxi_Game_using_Reinforcement_Q_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install cmake 'gym[atari]' scipy

In [ ]:
import gym

## The Problem Statement
There are 4 locations (labeled by different letters), and our job is to pick up the passenger at one location and drop him off at another. We receive +20 points for a successful drop-off and lose 1 point for every time-step it takes. There is also a 10 point penalty for illegal pick-up and drop-off actions.

In [ ]:
env = gym.make("Taxi-v3").env
env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



The core gym interface is `env`, which is the unified environment interface. The following are the env methods that would be quite helpful to us:

`env.reset`: Resets the environment and returns a random initial state.

`env.step(action)`: Step the environment by one timestep. Returns

1. observation: Observations of the environment
2. reward: If your action was beneficial or not
3. done: Indicates if we have successfully picked up and dropped off a passenger, also called one episode
4. info: Additional info such as performance and latency for debugging purposes

`env.render:` Renders one frame of the environment (helpful in visualizing the environment)

In [ ]:
env.reset() # reset environment to a new, random state
env.render()

print("Action Space {}".format(env.action_space))
print("State Space {}".format(env.observation_space))

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

Action Space Discrete(6)
State Space Discrete(500)


1. The filled square represents the taxi, which is yellow without a passenger and green with a passenger.
2. The pipe ("|") represents a wall which the taxi cannot cross.
3. R, G, Y, B are the possible pickup and destination locations. The blue letter represents the current passenger pick-up location, and the purple letter is the current destination.

Action space = { south, north, east, west. pickup, dropoff }

The 500 states correspond to a encoding of the taxi's location, the passenger's location, and the destination location.

In [ ]:
state = env.encode(3, 1, 2, 0) # (taxi row, taxi column, passenger index, destination index)
print("State:", state)

env.s = state #setting the environmntal state manually
env.render()

State: 328
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [ ]:
#prinitng the reward table
env.P[328]
#its structure is :{action: [(probability, nextstate, reward, done)]}.

{0: [(1.0, 428, -1, False)],
 1: [(1.0, 228, -1, False)],
 2: [(1.0, 348, -1, False)],
 3: [(1.0, 328, -1, False)],
 4: [(1.0, 328, -10, False)],
 5: [(1.0, 328, -10, False)]}

All the movement actions have a -1 reward and the pickup/dropoff actions have -10 reward in this particular state. If we are in a state where the taxi has a passenger and is on top of the right destination, we would see a reward of 20 at the dropoff action (5)

done is used to tell us when we have successfully dropped off a passenger in the right location. Each successfull dropoff is the end of an episode

# New section: Intro to Q-learning


Essentially, Q-learning lets the agent use the environment's rewards to learn, over time, the best action to take in a given state.

In our Taxi environment, we have the reward table, `P`, that the agent will learn from. It does thing by looking receiving a reward for taking an action in the current state, then updating a Q-value to remember if that action was beneficial.

The values store in the Q-table are called a Q-values, and they map to a `(state, action)` combination.

Breaking it down into steps, we get

1. Initialize the Q-table by all zeros.
2. Start exploring actions: For each state, select any one among all possible actions for the current state (S).
3. Travel to the next state (S') as a result of that action (a).
4. For all possible actions from the state (S') select the one with the highest Q-value.
5. Update Q-table values using the equation.
6. Set the next state as the current state.
7. If goal state is reached, then end and repeat the process.

In [ ]:
import numpy as np

#initializing the Q-table to a 500 * 6 matrix of zeros. 
q_table = np.zeros([env.observation_space.n, env.action_space.n])
q_table

array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.]])

In [19]:
"""Training the Agent"""

import random
from IPython.display import clear_output

#Hyperparameters
alpha = 0.1
gamma = 0.6
epsilon = 0.1

#for plotting metrics
all_epochs = []
all_penalties = []

for i in range(1,100001):
  state =env.reset()

  epochs,penalties, reward, = 0, 0, 0
  done = False

  while not done:
    if random.uniform(0,1) < epsilon:
      action = env.action_space.sample() # Explore action space

    else:
            action = np.argmax(q_table[state]) # Exploit learned values

    next_state, reward, done, info = env.step(action)
    old_value = q_table[state, action]
    next_max = np.max(q_table[next_state])

    new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
    q_table[state, action] = new_value

    if reward == -10:
            penalties += 1

    state = next_state
    epochs += 1
        
    if i % 100 == 0:
        clear_output(wait=True)
        print(f"Episode: {i}")

print("Training finished.\n")

Episode: 100000
Training finished.



Now that the Q-table has been established over 100,000 episodes, let's see what the Q-values are at our illustration's state:

In [56]:
q_table[328]

array([ -2.41696597,  -2.27325184,  -2.41045062,  -2.3624831 ,
       -11.32644396, -11.18962504])

The max Q-value is "north" (-1.971), so it looks like Q-learning has effectively learned the best action to take in our illustration's state!

# New section: evaluating the agent
Let's evaluate the performance of our agent. We don't need to explore actions any further, so now the next action is always selected using the best Q-value:

In [88]:
NewState = env.reset()
env.s = NewState
env.render()
("state:", NewState)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



('state:', 64)

In [89]:
env.P[64]

{0: [(1.0, 164, -1, False)],
 1: [(1.0, 64, -1, False)],
 2: [(1.0, 84, -1, False)],
 3: [(1.0, 44, -1, False)],
 4: [(1.0, 64, -10, False)],
 5: [(1.0, 64, -10, False)]}

In [90]:
"""Evaluate agent's performance after Q-learning"""

total_epochs, total_penalties = 0, 0
episodes = 1
state = NewState
frames = [] # for animation

for _ in range(episodes):
    epochs, penalties, reward = 0, 0, 0
    done = False
    
    while not done:
        action = np.argmax(q_table[state])
        state, reward, done, info = env.step(action)

        if reward == -10:
            penalties += 1

        # Put each rendered frame into dict for animation
        frames.append({
              'frame': env.render(),
              'state': state,
              'action': action,
              'reward': reward
              }
          )    

        epochs += 1

    total_penalties += penalties
    total_epochs += epochs

print(f"Results after {episodes} episodes:")
print(f"Average timesteps per episode: {total_epochs / episodes}")
print(f"Average penalties per episode: {total_penalties / episodes}")

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Pickup)
+---------+
|R: | : :G|
| : | : :_|
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
| : | :_: |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)
+---------+
|R: | : :G|
| : | : : |
| : : :_: |
| | : | : |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
| : | : : |
| : :_: : |
| | : | : |
|Y| : |B: |
+---------+
  (West)
+---------+
|R: | : :G|
| : | : : |
| :_: : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)
+---------+
|R: | : :G|
| :_| : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
+---------+
|R: | : :G|
|_: | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| 